<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1


    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.63
    Uninstalling yfinance-0.2.63:
      Successfully uninstalled yfinance-0.2.63
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-25 10:19:28
-------------------
qualified stocks: 93
with latest results: 29
still star stocks: 17


-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  35.47 K
Current:  1.34 C
-------------------
Today PnL: 1.17 L (0.88%)
Current PnL: -15.69 L (-10.94%)
CY Booked + Current PnL: -9.32 L (-6.5%)
-------------------
Total profit:  3.35 L
Total loss:  -19.04 L
-------------------
Total Booked + Current PnL: 17.66 L (15.15%)
Total Booked PnL: 33.35 L (28.62%)
Curr Year Booked PnL: 6.37 L (4.77%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.06 C
Est FTT PnL: 72.15 L (54.04%)
Deployed:  1.17 C
Current:  1.34 C
CAGR/XIRR %: 9.33%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,LICI,827.06,983.0,-3.65,H-LC,84.95,97886.0,12699.0,3367.0,0.50,14.91,3.44,18.85,35.0,3.77,0.74,30.33,XY25,NTT,INSURANCE
62,RELIANCE,1291.83,1526.0,7.63,X-LC,51.61,159979.0,19170.0,6351.0,1.16,13.61,3.97,18.13,17.0,3.02,1.20,26.32,XY25,NTT,REFINERIES
84,UJJIVANSFB,52.77,53.0,54.03,M-SC,88.17,133164.0,-9315.0,9934.0,1.21,-6.54,7.46,0.44,242.0,-0.94,1.00,57.17,OX40N,NTT,BANKS
89,VIPIND,488.80,489.0,-986.06,H-SC,97.85,84376.0,-10940.0,10977.0,-0.84,-11.48,13.01,0.04,153.0,-1.00,0.63,66.42,OX40N,NTT,MISC
3,ADANIPORTS,1103.69,1583.0,-1.49,M-LC,38.71,98654.0,20292.0,13743.0,2.56,25.90,13.93,43.43,77.0,1.48,0.74,32.59,XY24,NTT,MISC


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SAIL,130.64,228.00,96.05,M-MC,47.31,222775.0,-2187.0,169844.0,-1.05,-0.97,76.24,74.53,184.0,-0.01,1.67,29.34,XY24,BTT,STEEL
12,BANKINDIA,116.91,190.00,-33.09,M-MC,52.69,178270.0,-1538.0,113950.0,0.56,-0.86,63.92,62.52,170.0,-0.01,1.34,32.94,XR,NTT,BANKS
50,LALPATHLAB,2804.19,3545.00,-33.57,H-SC,25.81,152801.0,-1429.0,42173.0,1.49,-0.93,27.60,26.42,126.0,-0.03,1.15,20.50,X40N,NTT,HEALTHCARE
14,BERGEPAINT,561.33,680.00,-16.80,H-MC,56.99,228339.0,1000.0,47061.0,1.90,0.44,20.61,21.14,117.0,0.02,1.71,27.85,XY24,NTT,PAINTS
34,HEROMOTOCO,4311.81,6039.03,-0.95,H-MC,36.56,151112.0,199.0,60248.0,0.58,0.13,39.87,40.06,100.0,0.00,1.13,23.60,AR,ATH,AUTO


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,ALKYLAMINE,2347.98,4546.37,7.41,X-SC,100.00,100031.0,-932.0,95460.0,5.85,-0.92,95.43,93.63,60.0,-0.01,0.75,48.99,SR,ATH,CHEMICALS
41,INDIAMART,2327.09,4871.06,-46.61,H-SC,91.40,138929.0,15593.0,119243.0,5.07,12.64,85.83,109.32,122.0,0.13,1.04,37.25,AR,ATH,MISC
31,HATHWAY,19.74,31.40,1008.70,L-SC,65.59,63560.0,-15400.0,62041.0,4.82,-19.50,97.61,59.07,259.0,-0.25,0.48,27.53,XR,NTT,ENTERTAINMENT
25,DEN,52.79,75.00,109.84,L-SC,74.19,59040.0,-25424.0,60959.0,4.21,-30.10,103.25,42.07,256.0,-0.42,0.44,25.38,AR,NTT,ENTERTAINMENT
48,JPPOWER,18.73,26.20,-33.07,L-SC,96.77,136390.0,-5452.0,62017.0,3.74,-3.84,45.47,39.88,258.0,-0.09,1.02,38.22,XY24,NTT,POWER


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,DIXON,14367.00,18940.15,-14.08,H-LC,15.05,113488.0,-1448.0,38030.0,-2.21,-1.26,33.51,31.83,101.0,-0.04,0.85,30.07,X40N,ATH,IT
71,SIEMENS,4406.90,7969.85,-13.85,X-LC,19.35,77980.0,-32192.0,121267.0,-1.28,-29.22,155.51,80.85,11.0,-0.27,0.59,14.85,AR,ATH,ELECTRICAL
85,UNIONBANK,123.87,163.00,-6.44,M-LC,63.44,165127.0,24287.0,20212.0,-1.26,17.24,12.24,31.59,86.0,1.20,1.24,43.44,XY24,NTT,BANKS
65,SAIL,130.64,228.00,96.05,M-MC,47.31,222775.0,-2187.0,169844.0,-1.05,-0.97,76.24,74.53,184.0,-0.01,1.67,29.34,XY24,BTT,STEEL
89,VIPIND,488.80,489.00,-986.06,H-SC,97.85,84376.0,-10940.0,10977.0,-0.84,-11.48,13.01,0.04,153.0,-1.00,0.63,66.42,OX40N,NTT,MISC


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,UJJIVANSFB,52.77,53.0,54.03,M-SC,88.17,133164.0,-9315.0,9934.0,1.21,-6.54,7.46,0.44,242.0,-0.94,1.00,57.17,OX40N,NTT,BANKS
89,VIPIND,488.80,489.0,-986.06,H-SC,97.85,84376.0,-10940.0,10977.0,-0.84,-11.48,13.01,0.04,153.0,-1.00,0.63,66.42,OX40N,NTT,MISC
78,TATAELXSI,7332.28,7332.0,-16.82,X-MC,30.11,87360.0,-15292.0,15288.0,-0.41,-14.90,17.50,-0.00,57.0,-1.00,0.66,32.80,OX40N,NTT,IT
76,SYMPHONY,1306.42,1306.0,-33.17,M-SC,1.08,142397.0,-28744.0,28693.0,1.21,-16.80,20.15,-0.03,190.0,-1.00,1.07,2.34,OX40N,NTT,DURABLES
42,INDIGOPNTS,1407.73,1408.0,119.87,H-SC,89.25,144299.0,-30260.0,30288.0,0.08,-17.33,20.99,0.02,127.0,-1.00,1.08,25.13,OX40N,NTT,PAINTS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,INFY,1461.46,2275.00,-10.62,X-LC,54.84,286400.0,24799.0,120832.0,1.16,9.48,42.19,55.67,1.0,0.21,2.15,16.16,X40,BTT,IT
80,TCS,3794.03,4998.00,-18.41,X-LC,26.88,266347.0,-29587.0,123505.0,0.72,-10.00,46.37,31.73,2.0,-0.24,2.00,6.61,X40,BTT,IT
36,HINDUNILVR,2413.81,2723.00,-18.57,X-LC,21.51,255998.0,-14349.0,48972.0,1.00,-5.31,19.13,12.81,5.0,-0.29,1.92,7.06,XY25,NTT,FMCG
15,BRITANNIA,4983.33,6446.05,14.32,X-LC,59.14,261970.0,32737.0,34554.0,1.39,14.28,13.19,29.35,7.0,0.95,1.97,24.48,XY25,ATH,FMCG
57,NESTLEIND,2268.60,2755.00,-10.82,X-LC,33.33,283316.0,17890.0,39013.0,2.35,6.74,13.77,21.44,10.0,0.46,2.13,13.35,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ADANIPORTS,1103.69,1583.0,-1.49,M-LC,38.71,98654.0,20292.0,13743.0,2.56,25.9,13.93,43.43,77.0,1.48,0.74,32.59,XY24,NTT,MISC


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMS,3643.00,5250.99,7.56,H-SC,83.87,120826.0,18822.0,26207.0,2.09,18.45,21.69,44.14,125.0,0.72,0.91,37.69,X40N,ATH,MISC
34,HEROMOTOCO,4311.81,6039.03,-0.95,H-MC,36.56,151112.0,199.0,60248.0,0.58,0.13,39.87,40.06,100.0,0.00,1.13,23.60,AR,ATH,AUTO
3,ADANIPORTS,1103.69,1583.00,-1.49,M-LC,38.71,98654.0,20292.0,13743.0,2.56,25.90,13.93,43.43,77.0,1.48,0.74,32.59,XY24,NTT,MISC
92,WIPRO,243.46,420.00,-2.90,M-LC,79.57,166067.0,15122.0,94326.0,1.09,10.02,56.80,72.51,68.0,0.16,1.25,14.39,XR,NTT,IT
85,UNIONBANK,123.87,163.00,-6.44,M-LC,63.44,165127.0,24287.0,20212.0,-1.26,17.24,12.24,31.59,86.0,1.20,1.24,43.44,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,WHIRLPOOL,1167.49,2270.00,-31.00,M-SC,81.72,201316.0,29695.0,132365.0,-0.37,17.30,65.75,94.43,194.0,0.22,1.51,48.58,XR,NTT,DURABLES
92,WIPRO,243.46,420.00,-2.90,M-LC,79.57,166067.0,15122.0,94326.0,1.09,10.02,56.80,72.51,68.0,0.16,1.25,14.39,XR,NTT,IT
41,INDIAMART,2327.09,4871.06,-46.61,H-SC,91.40,138929.0,15593.0,119243.0,5.07,12.64,85.83,109.32,122.0,0.13,1.04,37.25,AR,ATH,MISC
56,NATIONALUM,189.63,247.44,-36.43,H-SC,66.67,101092.0,588.0,30055.0,0.52,0.59,29.73,30.49,97.0,0.02,0.76,33.18,X5K,ATH,METALS
34,HEROMOTOCO,4311.81,6039.03,-0.95,H-MC,36.56,151112.0,199.0,60248.0,0.58,0.13,39.87,40.06,100.0,0.00,1.13,23.60,AR,ATH,AUTO


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,59.14,80.00,-51.71,L-SC,95.70,72634.0,-40915.0,80965.0,1.34,-36.03,111.47,35.27,260.0,-0.51,0.55,77.77,XR,NTT,HOTELS
7,ASIANTILES,75.41,137.00,7350.00,M-SC,98.92,81731.0,-12079.0,88694.0,1.05,-12.88,108.52,81.67,238.0,-0.14,0.61,58.28,XR,NTT,CERAMICS
89,VIPIND,488.80,489.00,-986.06,H-SC,97.85,84376.0,-10940.0,10977.0,-0.84,-11.48,13.01,0.04,153.0,-1.00,0.63,66.42,OX40N,NTT,MISC
54,MASFIN,326.60,402.39,-18.70,H-SC,75.27,92580.0,-5400.0,28135.0,2.66,-5.51,30.39,23.21,141.0,-0.19,0.70,34.27,XR,ATH,FINANCE
52,LICI,827.06,983.00,-3.65,H-LC,84.95,97886.0,12699.0,3367.0,0.50,14.91,3.44,18.85,35.0,3.77,0.74,30.33,XY25,NTT,INSURANCE


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,ALKYLAMINE,2347.98,4546.37,7.41,X-SC,100.00,100031.0,-932.0,95460.0,5.85,-0.92,95.43,93.63,60.0,-0.01,0.75,48.99,SR,ATH,CHEMICALS
7,ASIANTILES,75.41,137.00,7350.00,M-SC,98.92,81731.0,-12079.0,88694.0,1.05,-12.88,108.52,81.67,238.0,-0.14,0.61,58.28,XR,NTT,CERAMICS
89,VIPIND,488.80,489.00,-986.06,H-SC,97.85,84376.0,-10940.0,10977.0,-0.84,-11.48,13.01,0.04,153.0,-1.00,0.63,66.42,OX40N,NTT,MISC
48,JPPOWER,18.73,26.20,-33.07,L-SC,96.77,136390.0,-5452.0,62017.0,3.74,-3.84,45.47,39.88,258.0,-0.09,1.02,38.22,XY24,NTT,POWER
22,COFFEEDAY,59.14,80.00,-51.71,L-SC,95.70,72634.0,-40915.0,80965.0,1.34,-36.03,111.47,35.27,260.0,-0.51,0.55,77.77,XR,NTT,HOTELS


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.22
1,25,42.46
2,50,71.25


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    38.78
LC    37.90
MC    21.56
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.20
XY25     14.13
X40      12.41
X40N     10.67
XR       10.39
OX40N     8.31
AR        7.92
X5K       2.23
X200      1.46
SR        1.24
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    19.57
X-LC    15.39
M-SC    13.93
H-LC    13.15
H-MC     8.46
M-LC     8.26
X-MC     7.53
M-MC     5.57
X-SC     2.79
L-SC     2.49
L-LC     1.10
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.41,-1.94,36.03
IT,12.00,-11.11,66.26
BANKS,7.96,-7.62,51.53
FINANCE,7.91,-14.52,55.97
MISC,6.72,-20.67,67.86
PAINTS,5.60,-16.72,39.72
ELECTRICAL,5.09,-3.78,48.72
HEALTHCARE,4.54,-6.67,37.83
AUTO,4.42,-14.74,65.19


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2773442.0,25
XR,1077493.0,13
AR,1015143.0,9
X40,664690.0,9
X40N,508220.0,10
XY25,508083.0,10
OX40N,321332.0,11
SR,178731.0,2
X5K,101931.0,3


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1890242.0,19
M-SC,1470400.0,18
X-LC,614953.0,9
M-MC,602887.0,5
H-LC,602406.0,12
H-MC,495649.0,7
X-MC,414174.0,6
M-LC,345472.0,6
L-SC,265982.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,872656.0,6
M-SC,XY24,595596.0,5
H-SC,AR,517637.0,3
M-MC,XY24,397703.0,3
M-SC,XR,336190.0,4
X-LC,X40,319665.0,3
H-LC,X40N,238080.0,5
M-LC,XY24,233814.0,4
H-MC,XY24,233718.0,2


In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 47.0 seconds
